# Auto / Manual RC Car Control

### -Control Raspberry Pi by Using Image AI and Http Protocol-

Hyeon Jin Yoo (from Department of biological science in Seoul National University)

코딩 기술서에서 사용된 방법론을 구현하기 위한 Python Code를 서술해두었으며, 코딩 기술서 내 목차 순서와 동일하게 구성되었으므로, 필요한 부분에 해당하는 코드를 해당 절에서 찾아 이용자의 환경에 맞추어 사용하면 된다.

## II. Automatically Controlling by Image AI

### 2.1. Real World Image Processing

1) Capture Images

라즈베리파이에 연동된 카메라에서 이미지를 캡처해 라즈베리파이 내 로컬 저장소에 이미지들을 자동 넘버링하여 저장하는 코드이다. 일정 시간마다 자동 캡처되도록 구성하였으며, 이미지 크기나 사진 캡처 주기는 원하는대로 조정해서 사용할 수 있다. 다음 셀의 코드르를 py파일로 저장하여 라즈베리파이 내에서 사용하도록 하자.

In [ ]:
import cv2
from time import sleep

cap = cv2.VideoCapture(0)
num = 1 # Image number starting point
interval = 0.3 # Can change capturing time interval
image_size = [480, 240] # Can change the image size

def getImg(num, size = image_size):
    ret, img = cap.read()
    while (cap.isOpened()):
        img = cv2.resize(img, (size[0],size[1]))
        cv2.imshow('IMG', img)
        cv2.imwrite('./%d.jpg' % num, img) # Can change directory
        sleep(interval)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
        return img
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    while True:
        img = getImg(num = num)
        num += 1
        cv2.waitKey(1)

2) Thresholding the Image

위 과정에서 수집했던 이미지 파일을 대상으로 라벨링 작업을 수행하기 전, AI가 이미지의 특징을 더 잘 파악할 수 있도록 Thresholding하는 과정이 필요하다. 우선, 이미지가 python 내에서 잘 불러와 지는지 확인하는 작업을 수행하도록 하자.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

img = mpimg.imread('1.jpg') # Can change image number and directory
plt.imshow(img)
plt.show()

이미지가 잘 불러와졌는지 확인이 되었다면 이제 본격적인 Thresholding 작업을 수행하자. 이 작업은 BGR, HSV 두 pixel value에서 모두 가능하며, Thresholding을 원하는 image 번호와 Thresholding 방식을 선택한 뒤, trackbar를 움직이면서 적절한 value의 범위를 찾아내자. 최대한 많은 이미지에 적용될 수 있는 value의 범위를 찾아 전처리를 해주고 이를 AI 지도 학습에 이용하도록 하자. 만일, image의 noise 등의 문제로 적절한 Thresholding이 어렵다면 다시 image collecting을 하는 것을 권장한다.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import numpy as np

image_number = 1 # Choose image number
values = 'BGR' # Choose between 'BGR' and 'HSV'

img = cv2.imread('%d.jpg' % image_number) # Can change directory
if values == 'HSV':
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
val_dict = {'BGR' : 255, "HSV" : 179}

def empty(a):
    pass

cv2.namedWindow(values)
cv2.resizeWindow(values, 640, 240)
cv2.createTrackbar("%s Min" %values[0], values, 0, val_dict[values], empty)
cv2.createTrackbar("%s Max" %values[0], values, val_dict[values], val_dict[values], empty)
cv2.createTrackbar("%s Min" %values[1], values, 0, 255, empty)
cv2.createTrackbar("%s Max" %values[1], values, 255, 255, empty)
cv2.createTrackbar("%s Min" %values[2], values, 0, 255, empty)
cv2.createTrackbar("%s Max" %values[2], values, 255, 255, empty)

while cv2.waitKey(1) != ord('q'): # If press 'q', can stop running and quit

    a_min = cv2.getTrackbarPos("%s Min" %values[0], values)
    a_max = cv2.getTrackbarPos("%s Max" %values[0], values)
    b_min = cv2.getTrackbarPos("%s Min" %values[1], values)
    b_max = cv2.getTrackbarPos("%s Max" %values[1], values)
    c_min = cv2.getTrackbarPos("%s Min" %values[2], values)
    c_max = cv2.getTrackbarPos("%s Max" %values[2], values)

    lower = np.array([a_min, b_min, c_min])
    upper = np.array([a_max, b_max, c_max])
    mask = cv2.inRange(img, lower, upper)
    result = cv2.bitwise_and(img, img, mask=mask)

    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    hStack = np.hstack([img, mask, result])
    cv2.imshow('Horizontal Stacking', hStack)

cv2.waitKey(0)
cv2.destroyAllWindows()

3) Preprocessing Data

수집한 이미지에 대해서 각 이미지가 좌, 전진, 우 방향 중 어느 방향으로 가는 것이 적절할지 라벨링을 먼저 수행해야한다. 다음 코드를 실행시켜 txt 파일을 만들고 각 번호의 이미지가 어느 방향인지 'l', 's', 'r' (각각 좌, 전진, 우)로 라벨링을 해두고 저장하자.

In [ ]:
num = 1 # Change to the number of images

f = open('label.txt','a')
for i in range(1, num + 1):
    f.write("%d \n" % i)
f.close()

라벨링을 종료하였으면, 위에서 정했던 Thresholding 기준을 가지고 최종 input data를 만들자.

In [ ]:
num = 1 # Change to the number of images
cut_offs = [60, 60, 60] # Change to the threshold values
values = 'BGR' # Choose between 'BGR' and 'HSV'

data = []

for i in range(1, num + 1):
    try:
        img = cv2.imread('%d.jpg' % i)
        if values == 'HSV':
            img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        lower = np.array([0, 0, 0]) # If designate the minimum cut-offs, change it
        upper = np.array(cut_offs)

        mask = cv2.inRange(img, lower, upper)
        result = cv2.bitwise_and(img, img, mask=mask)
        
        data.append(mask)
            
    except:
        pass

In [ ]:
f = open('label.txt', 'r')
labels = f.readlines()
f.close()

label_num = []

for label in labels:
    label_num.append(label.split()[1].replace('l', '0').replace('s', '1').replace('r', '2'))

label_num = list(map(int, label_num))

### 2.2. Supervised Learning by CNN

이제 AI 학습을 위한 데이터셋을 모두 생성하였으니, 학습을 수행해주면 된다. 학습 수행을 하기 전에 train, test dataset 분리를 진행하자.

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(data, label_num, test_size=0.2) # Can change test dataset size

image_size = [240, 480] # Choose the size of images

train_data = np.array(train_data).reshape(-1, image_size[0], image_size[1], 1)
train_label = np.array(train_label)
test_data = np.array(test_data).reshape(-1, image_size[0], image_size[1], 1)
test_label = np.array(test_label)

지도 학습은 CNN 모델을 통해서 이루어진다. 과적합 방지를 이용하여 모델 학습을 진행하고 학습이 종료되면 모델 저장까지 해보자.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import Flatten, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping

# Can change structures and hyperparameters of model
image_size = [240, 480] # Choose the size of images

model = Sequential()
model.add(Conv2D(16, (3, 3), padding='valid',
                  input_shape=(image_size[0], image_size[1], 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(16, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3), padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping() # Prevent overfitting
model.fit(train_data, train_label, epochs=10, validation_data=(test_data, test_label), callbacks=[early_stopping]) # Choose # of epochs
model.evaluate(test_data, test_label, verbose=2)

In [ ]:
# Save the final model
model.save("final_model.h5")

### 2.3. Hardware Controlling Strategy

이제 학습된 모델을 가지고 카메라 이미지를 통한 간이 자율주행을 직접 수행해볼 차례이다. 다음 파일을 py파일로 라즈베리파이 내에서 실행해보자. 단, 두 모터에 연결된 pin의 번호를 체크하고, 원하는 속도와 주행 중간 멈추는 시간을 설정해두자. 코드 앞부분에 위에서 구했던 여러 파라미터들의 값도 입력한 뒤 실행하도록 하자.

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
import time

from gpiozero import PWMLED
from gpiozero import LED

from time import sleep

### Check the gpio pin numbers that connect to motor driver ###

# Left wheel
PMA = PWMLED(2)
MA1 = LED(3)
MA2 = LED(4)

# Right wheel
PMB = PWMLED(22)
MB1 = LED(17)
MB2 = LED(27)

### Check the parameters ###

image_size = [240, 480] # Choose the size of images
cut_offs = [60, 60, 60] # Change to the threshold values
values = 'BGR' # Choose between 'BGR' and 'HSV'
speed = 0.3 # Choose the speed of car(between 0 and 1)
time_interval = {'turn' : 0.1, 'forward' : 0.25, 'stop' : 0.2} # Choose time intervals(second) for turn(left or right), forward and stop
run_time = 200 # Choose total running time of car(second)

### Driving ###

def Go(direction, speed, time):
    PMA.value = speed
    PMB.value = speed
    if direction == 's':
        MA1.on()
        MA2.off()
        MB1.on()
        MB2.off()
    elif direction == 'l':
        MA1.off()
        MA2.on()
        MB1.on()
        MB2.off()
    elif direction == 'r':
        MA1.on()
        MA2.off()
        MB1.off()
        MB2.on()
    sleep(time)
    
def stop():
    MA1.off()
    MA2.off()
    MB1.off()
    MB2.off()

cap = cv2.VideoCapture(0)
model = tf.keras.models.load_model("final_model.h5") # Can change name and directory of model
dir_list = ['l', 's', 'r']

start = time.time()
while True:
    ret, img = cap.read()
    
    img = cv2.resize(img, (image_size[0], image_size[1]))
    if values == 'HSV':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    lower = np.array([0, 0, 0]) # If designate the minimum cut-offs, change it
    upper = np.array(cut_offs)

    mask = cv2.inRange(img, lower, upper)
    result = cv2.bitwise_and(img, img, mask=mask)

    direction = np.argmax(model.predict(np.array(mask).reshape(-1, image_size[0], image_size[1], 1)))

    if direction != 1:
        Go(dir_list[direction], speed, time_interval['turn'])
    Go('s', speed, time_interval['forward'])
    stop()
    sleep(time_interval['stop'])
    if time.time() - start > run_time:
        stop()
        break

## III. Manually Controlling by Http Protocol

### 3.1. Making App to Controlling Raspberry Pi

이 파트에서는 Python Code 부분은 없으나 MIT App Inventor를 이용하여 코딩 기술서에 제시된 앱을 만드는 간단한 방법에 대하여 소개하고 Http 통신에 필요한 간단한 코드의 예시를 설명할 것이다. 우선, MIT App Inventor에 접속하여 새로운 프로젝트를 생성하고, 원하는 만큼 Screen 상에 Horizontal Arrangement를 배치한다. 그 후, 각 Horizontal Arrangement 내에 버튼을 배치하는데 좌하단의 Media 부분에 이미지를 첨부하여 원하는 모양으로 만들 수 있다. 그리고 우측의 메뉴바를 통해 글자 크기, 텍스트 내용, 버튼의 크기 등을 원하는대로 조정한다. 혼동을 피하기 위하여 각 버튼의 이름을 기능에 맞게 설정해주면 편리하다.

Web을 Screen 내에 Non-visible components로 설정해두는 작업이 중요한데, 이 Web의 주소는 http:// ip address of raspberry pi : port number로 해두어야만 한다. 여기까지 세팅을 해주면 기본적인 세팅은 끝나고 이제 라즈베리파이와의 통신을 위한 코드만 작성하면 되는데 우상단의 Blocks 파트로 넘어가서 다음 코드의 예시처럼 다른 버튼도 복사 붙여넣기를 해주면 된다. 자세한 예시는 코딩 기술서의 Fig.6(a)를 참고하면 된다.

In [ ]:
when 'button name' .Click
do  {set 'Web name' Url to 'Web address + :Get method name'}
    {call 'Web name' get}

### 3.2. Controlling Raspberry Pi by Http Protocol

3.1 절에서 만든 앱에서 보낸 Http 신호를 받아 하드웨어를 제어하는 코드이다. ip 주소 등을 현재 상황에 맞게 바꾸고 여러 파라미터를 조정하여 원하는 바대로 구현할 수 있다. 다음 파일의 파라미터를 조정하여 py파일로 저장한 뒤, 라즈베리파이 내에서 실행하고 같은 wifi 내에 연결된 기기에 설치된 앱을 이용하여 라즈베리파이를 제어해보자.

In [ ]:
import cv2
import tensorflow as tf
import numpy as np
import time

from gpiozero import LED, PWMLED
from time import sleep

from http.server import BaseHTTPRequestHandler, HTTPServer

### Check the gpio pin numbers that connect to motor driver ###

# Left wheel
PMA = PWMLED(2)
MA1 = LED(3)
MA2 = LED(4)

# Right wheel
PMB = PWMLED(22)
MB1 = LED(17)
MB2 = LED(27)

### Set ip address and port number of Raspberry Pi ###

ip_address = '192.168.43.60'
port = 8000

### Check the parameters ###

image_size = [240, 480] # Choose the size of images
cut_offs = [60, 60, 60] # Change to the threshold values
values = 'BGR' # Choose between 'BGR' and 'HSV'
speed = 0.3 # Choose the speed of car(between 0 and 1)
time_interval = {'turn' : 0.1, 'forward' : 0.25, 'stop' : 0.2} # Choose time intervals(second) for turn(left or right), forward and stop
epoch = 5 # Choose the number of epochs of auto moving
upper_bound = 0.5 # Upper bound of speed
lower_bound = 0.2 # Lower bound of speed
speed_delta = 0.05 # Amount of change of speed for one button click

def Go(direction):
    if direction == 's':
        MA1.on()
        MA2.off()
        MB1.on()
        MB2.off()
    elif direction == 'l':
        MA1.off()
        MA2.on()
        MB1.on()
        MB2.off()
    elif direction == 'r':
        MA1.on()
        MA2.off()
        MB1.off()
        MB2.on()
    elif direction == 'b':
        MA1.off()
        MA2.on()
        MB1.off()
        MB2.on()

def stop():
    MA1.off()
    MA2.off()
    MB1.off()
    MB2.off()

def set_speed(speed):
    PMA.value = speed
    PMB.value = speed    
    
cap = cv2.VideoCapture(0)
model = tf.keras.models.load_model("final_model.h5") # Can change name and directory of model
dir_list = ['l', 's', 'r']
AUTO = False
Dir = None
D_value = 0
Request = None

Clear_Http_request_in_the_server__Variable = None

class RequestHandler_httpd(BaseHTTPRequestHandler):
    def do_GET(self):
        global Request,Dir,D_value,Clear_Http_request_in_the_server__Variable, speed, AUTO, image_size, cut_offs, values
        global time_interval, epoch, upper_bound, lower_bound, speed_delta, dir_list, model
        messagetosend = bytes('',"utf")
        self.send_response(200)
        self.send_header('Content-Type', 'text/plain')
        self.send_header('Content-Length', len(messagetosend))
        self.end_headers()
        self.wfile.write(messagetosend)
        Request = self.requestline
        Request = Request[5 : int(len(Request)-9)]
        print(Request)

        if Request == 'Au': # Can change the name of request (recommended 2 characters word)
            AUTO = True

        if Request == 'Ma':
            AUTO = False

        if AUTO: # Auto mode
            ret, img = cap.read()

            img = cv2.resize(img, (image_size[0], image_size[1]))

            if values == 'HSV':
                img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

            lower = np.array([0, 0, 0]) # If designate the minimum cut-offs, change it
            upper = np.array(cut_offs)

            mask = cv2.inRange(img, lower, upper)
            result = cv2.bitwise_and(img, img, mask=mask)

            direction = np.argmax(model.predict(np.array(mask).reshape(-1, image_size[0], image_size[1], 1)))

            for i in range(epoch):
                set_speed(speed)
                if direction != 1:
                    Go(dir_list[direction], speed, time_interval['turn'])
                Go('s', speed, time_interval['forward'])
                stop()
                sleep(time_interval['stop'])

        else: # Manual mode
            if Request == 'Fo': # Can change the name of request (recommended 2 characters word)
            Go('s')

            if Request == 'Ba':
            Go('b')

            if Request == 'Ri':
            Go('r')

            if Request == 'Le':
            Go('l')

            if Request == 'St':
            stop()

            if (Request == 'Up') and (speed < upper_bound):
            speed += speed_delta
            set_speed(speed)

            if (Request == 'Do') and (speed > lower_bound):
            speed -= speed_delta
            set_speed(speed)
        return

server_address_httpd = (ip_address, port)
httpd = HTTPServer(server_address_httpd, RequestHandler_httpd)
print('Start server')
httpd.serve_forever()

문의 : hjy5405@naver.com